<a href="https://colab.research.google.com/github/nhahub/NHA-115/blob/main/IotLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run only if pyodbc driver is missing in Colab. May require runtime restart.
!apt-get update -qq
!apt-get install -y curl gnupg2 apt-transport-https
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update -qq
!ACCEPT_EULA=Y apt-get install -y msodbcsql18 unixodbc-dev


In [ ]:
# Run once in Colab
!pip install transformers accelerate torch bitsandbytes sentencepiece pyodbc pandas --quiet
print("Packages install step finished.")


In [ ]:
# Cell 1: Imports and configuration
import re, json, time
import pyodbc
import pandas as pd
from datetime import datetime, timezone
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import statistics

# ====== EDIT THESE ======
SERVER   = "iotsynaps.sql.azuresynapse.net"
DATABASE = "iotsqlpool"
USERNAME = "sqladminuser"
PASSWORD = "Babytools123"

IOT_TABLE    = "dbo.IoT_AirQuality"
REPORT_TABLE = "dbo.AI_Reports"

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
MODEL_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MAX_NEW_TOKENS = 300
TEMPERATURE = 0.4
RETRY_TEMPERATURE = 0.18

THRESH = {"pm25": 15.0, "pm10": 45.0, "no2": 25.0, "co2": 450.0}
GENERATED_BY = "Mistral-7B Q4"

print("Config set. Device:", MODEL_DEVICE)


In [ ]:
# Cell 2: Helper functions

def extract_number_from_text(s):
    if s is None: return None
    if not isinstance(s, str):
        try: return float(s)
        except: return None
    s_clean = s.replace("µ", "u").replace("μ", "u")
    m = re.search(r"[-+]?\d*\.\d+|\d+", s_clean)
    return float(m.group()) if m else None

def compute_stats_from_series(series):
    nums = [extract_number_from_text(x) for x in series.dropna().astype(str)]
    nums = [n for n in nums if n is not None]
    if not nums: return (None, None, None)
    return (round(statistics.mean(nums),2), round(max(nums),2), round(min(nums),2))

def compute_status(avg_pm25, avg_pm10, max_co2, max_no2):
    status = "Green"
    for key, val in [("pm25", avg_pm25), ("pm10", avg_pm10), ("co2", max_co2), ("no2", max_no2)]:
        if val is None: continue
        thr = THRESH.get(key)
        if thr is None: continue
        if val > 2 * thr: return "Red"
        if val > thr: status = "Yellow"
    return status

def detect_high_risk_regions(df):
    if not {'region','pm25','pm10'}.issubset(df.columns): return []
    df2 = df.copy()
    df2['_pm25_val'] = df2['pm25'].apply(extract_number_from_text)
    df2['_pm10_val'] = df2['pm10'].apply(extract_number_from_text)
    mask = (df2['_pm25_val'] > 50) | (df2['_pm10_val'] > 100)
    return df2.loc[mask, 'region'].dropna().unique().tolist()

def load_iot_data(hours_back=1, top_n=1000):
    conn_str = (
        f"Driver={{ODBC Driver 18 for SQL Server}};"
        f"Server={SERVER};Database={DATABASE};Uid={USERNAME};Pwd={PASSWORD};"
        f"Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
    )
    conn = pyodbc.connect(conn_str)
    query = f"""
    SELECT TOP ({top_n}) region, [timestamp], temperature, humidity, co2, no2, pm25, pm10
    FROM {IOT_TABLE}
    WHERE [timestamp] >= DATEADD(hour, -{int(hours_back)}, SYSUTCDATETIME())
    ORDER BY [timestamp] DESC
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def compute_metrics_and_prepare(df, hours_back):
    avg_pm25, max_pm25, min_pm25 = compute_stats_from_series(df['pm25']) if 'pm25' in df.columns else (None,None,None)
    avg_pm10, max_pm10, min_pm10 = compute_stats_from_series(df['pm10']) if 'pm10' in df.columns else (None,None,None)
    avg_co2, max_co2, min_co2   = compute_stats_from_series(df['co2']) if 'co2' in df.columns else (None,None,None)
    avg_no2, max_no2, min_no2   = compute_stats_from_series(df['no2']) if 'no2' in df.columns else (None,None,None)

    statistics_dict = {
        "avg_pm25": avg_pm25, "avg_pm10": avg_pm10, "max_co2": max_co2, "max_no2": max_no2,
        "min_pm25": min_pm25, "min_pm10": min_pm10, "min_co2": min_co2, "min_no2": min_no2,
    }

    payload = {
        "time_range": f"last {hours_back} hours",
        "statistics": statistics_dict,
        "samples": {col: df[col].dropna().astype(str).tolist()[:100] for col in ['pm25','pm10','co2','no2'] if col in df.columns},
        "high_risk_regions": detect_high_risk_regions(df),
        "status": compute_status(avg_pm25, avg_pm10, max_co2, max_no2)
    }
    return payload


In [ ]:
# Cell 3: Load tokenizer & model
print("Loading tokenizer and model (4-bit)...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    offload_folder="/tmp/offload"
)
print("Model loaded on device:", MODEL_DEVICE)


In [ ]:
# Cell 4: Build prompt & generate report
def build_prompt(payload):
    s = payload['statistics']
    samples = payload['samples']
    high_regions = payload['high_risk_regions']
    time_range = payload['time_range']

    return f"""
You are an expert environmental scientist. Produce a concise professional English summary and a final status (Green/Yellow/Red).
Use numeric stats for calculations and samples (which include units) as supporting evidence.

Time range: {time_range}

Instructions:
1) Output a short professional summary (4-6 sentences) with:
   - Overall status
   - Pollutants exceeding WHO limits
   - 2 actionable recommendations
2) Output EXACTLY one JSON block with keys:
   avg_pm25, avg_pm10, max_co2, max_no2, high_risk_regions, status

Numeric stats:
avg_pm25 = {s['avg_pm25']}
avg_pm10 = {s['avg_pm10']}
max_co2  = {s['max_co2']}
max_no2  = {s['max_no2']}

Sample PM2.5: {samples.get('pm25', [])[:10]}
Sample PM10: {samples.get('pm10', [])[:10]}

High risk regions: {high_regions}

Now generate the report.
"""

def generate_report(prompt_text, temp=TEMPERATURE):
    inputs = tokenizer(prompt_text, return_tensors="pt").to(MODEL_DEVICE)
    out = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        do_sample=True,
        temperature=temp
    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def produce_summary_and_json(payload):
    prompt_text = build_prompt(payload)
    report_raw = generate_report(prompt_text, temp=TEMPERATURE)

    # Retry if summary too short
    if len(report_raw.split("{")[0].strip()) < 20 or "You are an expert" in report_raw:
        report_raw = generate_report(prompt_text, temp=RETRY_TEMPERATURE)

    matches = re.findall(r"\{.*?\}", report_raw, re.DOTALL)
    parsed_json = None
    if matches:
        json_text = matches[-1].replace("'", '"')
        json_text = re.sub(r",\s*}", "}", json_text)
        json_text = re.sub(r",\s*]", "]", json_text)
        try: parsed_json = json.loads(json_text)
        except: parsed_json = None

    # Clean text summary
    if parsed_json:
        text_summary = report_raw.split(matches[-1])[0].strip()
    else:
        s = payload['statistics']
        text_summary = (
            f"Air quality status: {payload['status']}. "
            f"Avg PM2.5: {s.get('avg_pm25')}, PM10: {s.get('avg_pm10')}, "
            f"Max CO2: {s.get('max_co2')}, Max NO2: {s.get('max_no2')}. "
            f"High risk regions: {', '.join(payload['high_risk_regions']) or 'None'}."
        )
        parsed_json = {
            "avg_pm25": s.get('avg_pm25'),
            "avg_pm10": s.get('avg_pm10'),
            "max_co2": s.get('max_co2'),
            "max_no2": s.get('max_no2'),
            "high_risk_regions": payload['high_risk_regions'],
            "status": payload['status']
        }

    text_summary = re.sub(r"(?i)you are an expert.*?Now generate the report\.", "", text_summary, flags=re.DOTALL).strip()
    return text_summary, parsed_json, report_raw


In [ ]:
# Cell 5: Insert report into SQL
def insert_report_into_sql(ai_text_summary, stats_dict, high_regions, hours_back):
    conn_str = (
        f"Driver={{ODBC Driver 18 for SQL Server}};"
        f"Server={SERVER};Database={DATABASE};Uid={USERNAME};Pwd={PASSWORD};"
        f"Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    insert_sql = f"""
    INSERT INTO {REPORT_TABLE}
    (report_timestamp, ai_summary, avg_pm25, avg_pm10, max_co2, max_no2, high_risk_regions, generated_by, source_data_range)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(
        insert_sql,
        datetime.now(timezone.utc),
        ai_text_summary,
        stats_dict.get('avg_pm25'),
        stats_dict.get('avg_pm10'),
        stats_dict.get('max_co2'),
        stats_dict.get('max_no2'),
        json.dumps(high_regions, ensure_ascii=False),
        GENERATED_BY,
        f"last {int(hours_back)} hours"
    )
    conn.commit()
    cursor.close()
    conn.close()
    print("✅ Report inserted into", REPORT_TABLE)

def show_printable_results(text_summary, parsed_json):
    print("\n\n=== HUMAN SUMMARY ===\n")
    print(text_summary)
    print("\n\n=== PARSED JSON ===\n")
    print(json.dumps(parsed_json, indent=2, ensure_ascii=False))


In [ ]:
# Cell 6: Run full AI report
def run_ai_report(hours_back=1):
    print(f"Starting AI report for last {hours_back} hours...")
    df = load_iot_data(hours_back=hours_back)
    if df.empty:
        print("❌ No data found for this period.")
        return
    display(df.head())
    payload = compute_metrics_and_prepare(df, hours_back)
    print("\nComputed statistics:", payload['statistics'])
    print("High-risk regions:", payload['high_risk_regions'])
    print("Status:", payload['status'])

    text_summary, parsed_json, raw = produce_summary_and_json(payload)

    if parsed_json is None:
        print("⚠️ Warning: fallback JSON used.")
        s = payload['statistics']
        parsed_json = {
            "avg_pm25": s.get('avg_pm25'),
            "avg_pm10": s.get('avg_pm10'),
            "max_co2": s.get('max_co2'),
            "max_no2": s.get('max_no2'),
            "high_risk_regions": payload['high_risk_regions'],
            "status": payload['status']
        }

    show_printable_results(text_summary, parsed_json)
    insert_report_into_sql(text_summary, parsed_json, parsed_json.get('high_risk_regions', []), hours_back)
    print("\nFull raw model output preview:\n")
    print(raw[:1500], "...\n")

# Example run
run_ai_report(hours_back=3)
